# Data Compressed - rar.exe
An adversary may compress data (e.g., sensitive documents) that is collected prior to exfiltration in order to make it portable and minimize the amount of data sent over the network

## Rule Content
```
- title: Data Compressed - rar.exe
  id: 6f3e2987-db24-4c78-a860-b4f4095a7095
  status: experimental
  description: An adversary may compress data (e.g., sensitive documents) that is
    collected prior to exfiltration in order to make it portable and minimize the
    amount of data sent over the network
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1002/T1002.yaml
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image|endswith: \rar.exe
      CommandLine|contains|all:
      - ' a '
      - -r
    condition: selection
  fields:
  - Image
  - CommandLine
  - User
  - LogonGuid
  - Hashes
  - ParentProcessGuid
  - ParentCommandLine
  falsepositives:
  - highly likely if rar is default archiver in the monitored environment
  level: low
  tags:
  - attack.exfiltration
  - attack.t1002

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\rar.exe AND process_command_line.keyword:*\ a\ * AND process_command_line.keyword:*\-r*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()